In [50]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

Load & Preprocess Data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [13]:
BATCH_SIZE = 5

In [14]:
training_data_generator = ImageDataGenerator(rescale=1.0/255, zoom_range=0.2, rotation_range=15, width_shift_range=0.05, height_shift_range=0.05)
training_iterator = training_data_generator.flow_from_directory("/content/drive/My Drive/Covid19-dataset/train", class_mode="categorical", color_mode="grayscale", batch_size=BATCH_SIZE)

Found 251 images belonging to 3 classes.


In [15]:
validation_data_generator = ImageDataGenerator(rescale=1.0/255)
validation_iterator = validation_data_generator.flow_from_directory("/content/drive/My Drive/Covid19-dataset/test", class_mode="categorical", color_mode="grayscale")

Found 66 images belonging to 3 classes.


Model Creation

In [46]:
model = tf.keras.Sequential()
model.add(layers.Input(shape=(256, 256, 1)))
model.add(layers.Conv2D(5, 5, strides=3, padding="valid", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(5,5), strides=(3,3), padding="valid"))
model.add(layers.Conv2D(5, 3, strides=3, padding="valid", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))
#model.add(layers.Conv2D(2, 1, strides=1, padding="valid", activation="relu"))
#model.add(layers.MaxPooling2D(pool_size=(1,1), strides=(3,3), padding="valid"))
model.add(layers.Flatten())
model.add(tf.keras.layers.Dense(8, activation="relu"))
model.add(layers.Dense(3, activation="softmax"))

model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_48 (Conv2D)           (None, 84, 84, 5)         130       
_________________________________________________________________
max_pooling2d_44 (MaxPooling (None, 27, 27, 5)         0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 9, 9, 5)           230       
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 4, 4, 5)           0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 80)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 8)                 648       
_________________________________________________________________
dense_21 (Dense)             (None, 3)               

In [47]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()])

In [48]:
model.fit(training_iterator, steps_per_epoch=training_iterator.samples/BATCH_SIZE, epochs=5, validation_data = validation_iterator, validation_steps=validation_iterator.samples/BATCH_SIZE)

Epoch 1/5
51/50 [==============================] - 4s 78ms/step - loss: 1.0954 - categorical_accuracy: 0.4303 - auc_3: 0.5655 - val_loss: 1.0907 - val_categorical_accuracy: 0.3939 - val_auc_3: 0.5626
Epoch 2/5
51/50 [==============================] - 3s 63ms/step - loss: 1.0523 - categorical_accuracy: 0.4422 - auc_3: 0.6473
Epoch 3/5
51/50 [==============================] - 3s 62ms/step - loss: 0.8347 - categorical_accuracy: 0.6056 - auc_3: 0.8014
Epoch 4/5
51/50 [==============================] - 3s 62ms/step - loss: 0.7796 - categorical_accuracy: 0.6693 - auc_3: 0.8361
Epoch 5/5
51/50 [==============================] - 3s 63ms/step - loss: 0.6929 - categorical_accuracy: 0.6853 - auc_3: 0.8682


In [52]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [53]:
predictions = model.predict(validation_iterator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = validation_iterator.classes
class_labels = list(validation_iterator.class_indices.keys())
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)   

              precision    recall  f1-score   support

       Covid       0.48      0.38      0.43        26
      Normal       0.33      0.70      0.45        20
   Pneumonia       0.00      0.00      0.00        20

    accuracy                           0.36        66
   macro avg       0.27      0.36      0.29        66
weighted avg       0.29      0.36      0.30        66



In [54]:
cm=confusion_matrix(true_classes,predicted_classes)
print(cm)

[[10 13  3]
 [ 6 14  0]
 [ 5 15  0]]
